<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/04_bodypix_evaluador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
================================================================================
EVALUADOR BODYPIX - NOTEBOOK 2
================================================================================
Sistema de evaluacion para modelos BodyPix con estructura estandarizada
para comparacion de modelos de segmentacion de personas en fotografia de retrato.

CARACTERISTICAS PRINCIPALES:
- Procesamiento incremental con sistema de checkpoint robusto
- Gestion optima de memoria para Google Colab gratuito
- Guardado automatico despues de cada imagen procesada
- 3 modelos MobileNetV1 con multiplicadores 0.50, 0.75, 1.00
- 4 configuraciones de umbral de segmentacion
- Almacenamiento NPZ por fotografia con mascaras de probabilidad completas
- Segmentacion de partes del cuerpo (caracteristica unica de BodyPix)
- JSON ligero con metricas de rendimiento por umbral
- Visualizaciones estandarizadas

MODELOS DISPONIBLES:
1. MobileNetV1 0.50: Arquitectura ligera optimizada para velocidad
2. MobileNetV1 0.75: Balance entre velocidad y calidad de segmentacion
3. MobileNetV1 1.00: Maxima calidad de segmentacion disponible

CONFIGURACIONES DE UMBRAL:
Las configuraciones evaluan multiples umbrales de segmentacion para analizar
la sensibilidad del modelo a diferentes niveles de confianza:

1. ultra_sensible: [0.3, 0.4, 0.5] - Maxima deteccion de pixels
2. sensibilidad_alta: [0.4, 0.5, 0.6] - Alta sensibilidad
3. sensibilidad_media: [0.5, 0.65, 0.8] - Balance precision-recall
4. baja_sensibilidad: [0.7, 0.8, 0.9] - Alta confianza

ALMACENAMIENTO NPZ (por fotografia):
BodyPix genera mascaras de segmentacion continuas (valores 0-1) que representan
la probabilidad de que cada pixel pertenezca a una persona, y adicionalmente
puede segmentar partes del cuerpo. Se almacena:

Informacion de segmentacion de persona:
- mascara_probabilidad: Array (H, W) con valores 0.0-1.0 continuos
- mascaras_binarias_por_umbral: Dict con mascaras binarias para cada umbral
- dimensiones: Shape de la imagen procesada
- modelo_info: Metadatos del modelo usado

Informacion de partes del cuerpo (CARACTERISTICA UNICA de BodyPix):
- mascara_partes: Array (H, W) con IDs de partes 0-23, o -1 para fondo
- grupo_cara: Mascara binaria agrupada de cara
- grupo_torso: Mascara binaria agrupada de torso
- grupo_brazos: Mascara binaria agrupada de brazos
- grupo_manos: Mascara binaria agrupada de manos
- grupo_piernas: Mascara binaria agrupada de piernas
- grupo_pies: Mascara binaria agrupada de pies

NOTA SOBRE PARTES DEL CUERPO:
La segmentacion de partes del cuerpo es una caracteristica UNICA de BodyPix
que ningun otro modelo del TFM proporciona. Se almacena de forma LIGERA:
- La mascara de partes es int8 (mismo tamaño que la imagen)
- Se pre-calculan 6 grupos semanticos para fotografia de retrato
- Incremento de tamaño: ~7% adicional en NPZ comprimido
- Valor para el TFM: Permite analisis especificos de composicion de retratos

DIFERENCIAS CON OTROS MODELOS:
A diferencia de modelos de deteccion de instancias (YOLO, Mask2Former) o
generacion automatica (SAM2), BodyPix es un modelo de segmentacion semantica
que genera una unica mascara de probabilidad por imagen. No detecta instancias
individuales ni proporciona bounding boxes nativamente.

ESTRUCTURA DE SALIDA:
/TFM/2_Modelos/bodypix/{modelo}/{config}/resultados/
├── json/                           # Metricas por fotografia y umbral
├── mascaras/                       # NPZ con mascaras de probabilidad y partes
├── visualizaciones/                # Visualizaciones por fotografia
└── checkpoint.json                 # Estado del procesamiento

Entrada: Imagenes desde /TFM/0_Imagenes/
Salida: JSON, mascaras NPZ y visualizaciones en estructura organizada

Referencias:
- Papandreou et al. (2018) "PersonLab: Person Pose Estimation and Instance
  Segmentation with a Bottom-Up, Part-Based, Geometric Embedding Model"
- TensorFlow.js BodyPix: https://github.com/tensorflow/tfjs-models/tree/master/body-pix

Autor: Jesus L.
Proyecto: TFM - Evaluacion Comparativa de Tecnicas de Segmentacion
Universidad: Universidad Oberta de Cataluna (UOC)
Fecha: Octubre 2025
================================================================================

"""

'\n================================================================================\nEVALUADOR BODYPIX - NOTEBOOK 2\n================================================================================\nSistema de evaluacion para modelos BodyPix con estructura estandarizada\npara comparacion de modelos de segmentacion de personas en fotografia de retrato.\n\nCARACTERISTICAS PRINCIPALES:\n- Procesamiento incremental con sistema de checkpoint robusto\n- Gestion optima de memoria para Google Colab gratuito\n- Guardado automatico despues de cada imagen procesada\n- 3 modelos MobileNetV1 con multiplicadores 0.50, 0.75, 1.00\n- 4 configuraciones de umbral de segmentacion\n- Almacenamiento NPZ por fotografia con mascaras de probabilidad completas\n- Segmentacion de partes del cuerpo (caracteristica unica de BodyPix)\n- JSON ligero con metricas de rendimiento por umbral\n- Visualizaciones estandarizadas\n\nMODELOS DISPONIBLES:\n1. MobileNetV1 0.50: Arquitectura ligera optimizada para velocidad\n2.

In [2]:
# =============================================================================
# INSTALACION DE DEPENDENCIAS (EJECUTAR EN COLAB)
# =============================================================================

import subprocess
import sys

print("="*80)
print("INSTALANDO DEPENDENCIAS BODYPIX")
print("="*80)

# Orden correcto de instalacion para evitar conflictos
print("\n[1/5] Actualizando packaging...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "packaging>=24.2.0"])

print("[2/5] Instalando TensorFlow...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "tensorflow"])

print("[3/5] Instalando tfjs-graph-converter...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "tfjs-graph-converter"])

print("[4/5] Instalando tf-bodypix...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "tf-bodypix"])

print("[5/5] Instalando librerias de procesamiento...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "opencv-python", "Pillow", "matplotlib"])

print("\n" + "="*80)
print("DEPENDENCIAS INSTALADAS CORRECTAMENTE")
print("="*80 + "\n")

# =============================================================================
# MONTAR GOOGLE DRIVE
# =============================================================================

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    IN_COLAB = True
    print("Google Drive montado correctamente\n")
except:
    IN_COLAB = False
    print("Entorno fuera de Google Colab\n")

INSTALANDO DEPENDENCIAS BODYPIX

[1/5] Actualizando packaging...
[2/5] Instalando TensorFlow...
[3/5] Instalando tfjs-graph-converter...
[4/5] Instalando tf-bodypix...
[5/5] Instalando librerias de procesamiento...

DEPENDENCIAS INSTALADAS CORRECTAMENTE

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado correctamente



In [3]:
# =============================================================================
# IMPORTACIONES
# =============================================================================

import os
import gc
import time
import json
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, asdict, field

import numpy as np
import tensorflow as tf
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from tf_bodypix.api import download_model, load_model, BodyPixModelPaths

warnings.filterwarnings('ignore')

# Configurar TensorFlow para crecimiento dinamico de memoria GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU configurada: {len(gpus)} dispositivo(s) disponible(s)")
    except RuntimeError as e:
        print(f"Error configurando GPU: {e}")
else:
    print("Ejecutando en CPU")

print("Librerias importadas correctamente\n")

Ejecutando en CPU
Librerias importadas correctamente



In [4]:
# =============================================================================
# CONFIGURACION
# =============================================================================

@dataclass
class ModeloBodyPix:
    """Informacion de un modelo BodyPix"""
    nombre: str
    nombre_corto: str
    path: Any  # BodyPixModelPaths enum
    arquitectura: str
    multiplicador: float
    descripcion: str

MODELOS_DISPONIBLES = {
    'mobilenet_v1_050': ModeloBodyPix(
        nombre='MobileNetV1 0.50',
        nombre_corto='mobilenet_v1_050',
        path=BodyPixModelPaths.MOBILENET_FLOAT_50_STRIDE_16,
        arquitectura='MobileNetV1',
        multiplicador=0.50,
        descripcion='Modelo ultraligero optimizado para velocidad'
    ),
    'mobilenet_v1_075': ModeloBodyPix(
        nombre='MobileNetV1 0.75',
        nombre_corto='mobilenet_v1_075',
        path=BodyPixModelPaths.MOBILENET_FLOAT_75_STRIDE_16,
        arquitectura='MobileNetV1',
        multiplicador=0.75,
        descripcion='Balance entre velocidad y calidad de segmentacion'
    ),

    #    'resnet50': ModeloBodyPix(
    #     nombre='ResNet50',
    #     nombre_corto='resnet50',
    #     arquitectura='ResNet50',
    #     multiplicador=1.0,
    #     path=BodyPixModelPaths.RESNET50_FLOAT_STRIDE_16,
    #     descripcion='Maxima calidad de segmentacion disponible'
    # )
    # 'mobilenet_v1_100': ModeloBodyPix(
    #     nombre='MobileNetV1 1.00',
    #     nombre_corto='mobilenet_v1_100',
    #     path=BodyPixModelPaths.MOBILENET_FLOAT_100_STRIDE_16,
    #     arquitectura='MobileNetV1',
    #     multiplicador=1.00,
    #     descripcion='Maxima calidad de segmentacion disponible'
    # )
}

@dataclass
class ConfiguracionUmbrales:
    """
    Configuracion de umbrales de segmentacion para BodyPix.

    Los umbrales controlan que pixels se consideran como persona basandose
    en la probabilidad de segmentacion (valores 0.0 a 1.0).
    """
    nombre: str
    valores: List[float]
    descripcion: str

    def __post_init__(self):
        """Validacion de parametros"""
        if not all(0.0 <= v <= 1.0 for v in self.valores):
            raise ValueError("Los valores de umbral deben estar entre 0.0 y 1.0")

CONFIGURACIONES_UMBRALES = {
    'ultra_sensible': ConfiguracionUmbrales(
        nombre='ultra_sensible',
        valores=[0.10, 0.15, 0.20],
        descripcion='Maxima captura de la persona - Incluye bordes suaves (~55-66% area)'
    ),
    'sensibilidad_alta': ConfiguracionUmbrales(
        nombre='sensibilidad_alta',
        valores=[0.15, 0.20, 0.25],
        descripcion='Alta sensibilidad - Balance captura/precision (~50-60% area)'
    ),
    'sensibilidad_media': ConfiguracionUmbrales(
        nombre='sensibilidad_media',
        valores=[0.20, 0.30, 0.40],
        descripcion='Configuracion estandar para retrato - Precision optima (~40-55% area)'
    ),
    'baja_sensibilidad': ConfiguracionUmbrales(
        nombre='baja_sensibilidad',
        valores=[0.30, 0.40, 0.50],
        descripcion='Alta precision - Solo regiones de maxima confianza (~35-47% area)'
    )
}

@dataclass
class ConfiguracionEvaluacion:
    """Configuracion principal del sistema de evaluacion"""

    # Modelos a evaluar
    modelos_evaluar: List[str] = None  # None = todos los modelos

    # Configuraciones de umbrales a evaluar
    configs_umbrales: List[str] = None  # None = ['sensibilidad_media']

    # Rutas del sistema
    ruta_base: Path = Path("/content/drive/MyDrive/TFM")
    ruta_imagenes: Path = None
    ruta_salida: Path = None

    # Parametros de procesamiento
    max_dimension: int = 1024  # Dimension maxima para procesamiento
    pausa_entre_imagenes: float = 2.0  # Segundos entre imagenes
    pausa_entre_modelos: float = 5.0   # Segundos entre modelos

    def __post_init__(self):
        """Inicializacion de rutas por defecto"""
        if self.ruta_imagenes is None:
            self.ruta_imagenes = self.ruta_base / "0_Imagenes"
        if self.ruta_salida is None:
            self.ruta_salida = self.ruta_base / "2_Modelos" / "bodypix"
        if self.modelos_evaluar is None:
            self.modelos_evaluar = list(MODELOS_DISPONIBLES.keys())
        if self.configs_umbrales is None:
            self.configs_umbrales = ['sensibilidad_media']

In [5]:
# =============================================================================
# UTILIDADES
# =============================================================================

class Utilidades:
    """Utilidades para manejo de imagenes y archivos"""

    @staticmethod
    def cargar_imagen(ruta_imagen: Path, max_dimension: int = None) -> Tuple[np.ndarray, Dict]:
        """
        Carga una imagen y la convierte a RGB.

        CRÍTICO: BodyPix funciona MEJOR con imágenes grandes.
        NO redimensionar agresivamente. Solo si es absolutamente necesario.

        Args:
            ruta_imagen: Path a la imagen
            max_dimension: Dimensión máxima (usar None o valor alto tipo 4096)

        Returns:
            Tuple de (img_array_RGB, metadata)
        """
        # Cargar con OpenCV (BGR por defecto)
        img_bgr = cv2.imread(str(ruta_imagen))

        if img_bgr is None:
            raise ValueError(f"No se pudo cargar la imagen: {ruta_imagen}")

        # CRÍTICO: Convertir BGR -> RGB
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

        # Obtener dimensiones originales
        alto_orig, ancho_orig = img_rgb.shape[:2]

        # Redimensionar SOLO si es excesivamente grande Y max_dimension está definido
        if max_dimension and max(ancho_orig, alto_orig) > max_dimension:
            ratio = max_dimension / max(ancho_orig, alto_orig)
            nuevo_ancho = int(ancho_orig * ratio)
            nuevo_alto = int(alto_orig * ratio)
            img_rgb = cv2.resize(
                img_rgb,
                (nuevo_ancho, nuevo_alto),
                interpolation=cv2.INTER_AREA
            )
            redimensionado = True
        else:
            redimensionado = False

        # Metadata
        alto_proc, ancho_proc = img_rgb.shape[:2]
        info = {
            'ancho_original': ancho_orig,
            'alto_original': alto_orig,
            'ancho_procesado': ancho_proc,
            'alto_procesado': alto_proc,
            'redimensionado': redimensionado,
            'factor_escala': ancho_proc / ancho_orig if redimensionado else 1.0
        }

        return img_rgb, info

    @staticmethod
    def liberar_memoria():
        """
        Libera memoria y limpia sesión de TensorFlow/Keras.
        """
        import gc
        gc.collect()

        try:
            import tensorflow as tf
            if tf.config.list_physical_devices('GPU'):
                tf.keras.backend.clear_session()
        except:
            pass

    @staticmethod
    def guardar_json(datos: Dict, ruta: Path):
        """
        Guarda un diccionario como archivo JSON.
        """
        ruta.parent.mkdir(parents=True, exist_ok=True)

        with open(ruta, 'w', encoding='utf-8') as f:
            json.dump(datos, f, indent=2, ensure_ascii=False)

    @staticmethod
    def calcular_hash_imagen(ruta: Path) -> str:
        """
        Calcula hash MD5 de una imagen.
        """
        import hashlib
        with open(ruta, 'rb') as f:
            return hashlib.md5(f.read()).hexdigest()[:16]

In [6]:
# =============================================================================
# GESTOR DE CHECKPOINT
# =============================================================================

class GestorCheckpoint:
    """
    Gestiona el estado del procesamiento para permitir reanudacion tras
    interrupciones. Mantiene registro de imagenes completadas por combinacion
    de modelo y configuracion de umbrales.
    """

    def __init__(self, ruta_checkpoint: Path):
        self.ruta_checkpoint = ruta_checkpoint
        self.checkpoint = self._cargar()

    def _cargar(self) -> Dict:
        """Carga checkpoint existente o crea uno nuevo"""
        if self.ruta_checkpoint.exists():
            with open(self.ruta_checkpoint, 'r') as f:
                return json.load(f)
        return {
            'imagenes_completadas': [],
            'fecha_inicio': datetime.now().isoformat()
        }

    def obtener_pendientes(self, todas_imagenes: List[str]) -> set:
        """Retorna conjunto de imagenes pendientes de procesar"""
        completadas = set(self.checkpoint.get('imagenes_completadas', []))
        return set(todas_imagenes) - completadas

    def marcar_completada(self, nombre_imagen: str):
        """Marca una imagen como completada y persiste el checkpoint"""
        if nombre_imagen not in self.checkpoint['imagenes_completadas']:
            self.checkpoint['imagenes_completadas'].append(nombre_imagen)
            self.checkpoint['ultima_actualizacion'] = datetime.now().isoformat()
            Utilidades.guardar_json(self.checkpoint, self.ruta_checkpoint)

In [7]:
class GestorMascaras:
    """
    Gestor de almacenamiento de mascaras siguiendo patron estandar.

    MODIFICADO para consistencia con otros modelos del TFM:
    - Guarda UN archivo NPZ por imagen (sin sufijo de umbral)
    - El umbral esta implicito en la estructura de directorios
    - Nomenclatura: {foto_id}.npz con key 'person_mask'

    Estructura de salida:
    bodypix/modelo/config_T{umbral}/mascaras/{foto_id}.npz

    Contenido del NPZ:
    - person_mask: Mascara binaria principal (H, W) uint8
    - probability_mask: Mascara de probabilidad continua (H, W) float32
    - threshold_value: Valor del umbral usado (escalar)
    - body_parts_mask: IDs de partes del cuerpo (H, W) int8 (opcional)
    - grupo_{nombre}: Mascaras agrupadas uint8 (opcional)
    """

    GRUPOS_PARTES_CUERPO = {
        'cara': [0, 1],
        'torso': [2, 3, 4, 5, 12, 13],
        'brazos': [6, 7, 8, 9],
        'manos': [10, 11],
        'piernas': [14, 15, 16, 17, 18, 19, 20, 21],
        'pies': [22, 23]
    }

    def __init__(self, directorio_mascaras: Path):
        """
        Inicializa el gestor.

        Args:
            directorio_mascaras: Directorio donde guardar las mascaras
        """
        self.directorio_mascaras = Path(directorio_mascaras)
        self.directorio_mascaras.mkdir(parents=True, exist_ok=True)

    def guardar_mascaras(
        self,
        mascara_probabilidad: np.ndarray,
        umbrales: List[float],
        nombre_imagen: str,
        metadatos: Dict,
        mascara_partes: Optional[np.ndarray] = None,
        masks_por_umbral: Optional[Dict[float, np.ndarray]] = None
    ) -> str:
        """
        Guarda mascara en formato NPZ (un archivo por imagen).

        MODIFICADO: El nombre del archivo NO incluye el umbral.
        El umbral esta en la estructura de directorios y en los metadatos.

        Args:
            mascara_probabilidad: Mascara de probabilidad (H, W) float32
            umbrales: Lista con UN solo umbral
            nombre_imagen: Nombre de la imagen (ej: "_DSC0036.jpg")
            metadatos: Metadatos del modelo
            mascara_partes: Mascara de partes del cuerpo (H, W) int8
            masks_por_umbral: Dict con mascara para el umbral

        Returns:
            Ruta al archivo NPZ guardado

        Raises:
            ValueError: Si se reciben multiples umbrales
        """
        if len(umbrales) != 1:
            raise ValueError(
                f"GestorMascaras espera exactamente 1 umbral, "
                f"se recibieron {len(umbrales)}: {umbrales}"
            )

        umbral = umbrales[0]
        foto_id = Path(nombre_imagen).stem

        # Nombre simple sin sufijo de umbral
        nombre_npz = f"{foto_id}.npz"
        ruta_completa = self.directorio_mascaras / nombre_npz

        # Obtener mascara binaria para este umbral
        if masks_por_umbral is not None and umbral in masks_por_umbral:
            mascara_continua = masks_por_umbral[umbral]
            mascara_binaria = (mascara_continua > 0.5).astype(np.uint8)
        else:
            mascara_binaria = (mascara_probabilidad >= umbral).astype(np.uint8)

        # Construir diccionario NPZ
        datos_npz = {
            # Key principal compatible con indice maestro
            'person_mask': mascara_binaria,

            # Mascara de probabilidad continua
            'probability_mask': mascara_probabilidad.astype(np.float32),

            # Valor del umbral usado
            'threshold_value': np.array([umbral], dtype=np.float32)
        }

        # Agregar partes del cuerpo si estan disponibles
        if mascara_partes is not None:
            datos_npz['body_parts_mask'] = mascara_partes.astype(np.int8)

            for nombre_grupo, ids_partes in self.GRUPOS_PARTES_CUERPO.items():
                mascara_grupo = np.isin(mascara_partes, ids_partes).astype(np.uint8)
                datos_npz[f'grupo_{nombre_grupo}'] = mascara_grupo

        # Guardar archivo NPZ comprimido
        np.savez_compressed(ruta_completa, **datos_npz)

        return str(ruta_completa)

    def cargar_mascaras(self, ruta_archivo: Path) -> Dict:
        """
        Carga mascaras desde un archivo NPZ.

        Args:
            ruta_archivo: Path al archivo NPZ

        Returns:
            Diccionario con mascaras y metadatos

        Raises:
            FileNotFoundError: Si el archivo no existe
        """
        if not Path(ruta_archivo).exists():
            raise FileNotFoundError(f"Archivo no encontrado: {ruta_archivo}")

        data = np.load(ruta_archivo)

        resultado = {
            'person_mask': data['person_mask'],
            'probability_mask': data['probability_mask'] if 'probability_mask' in data else None,
            'threshold_value': float(data['threshold_value'][0]) if 'threshold_value' in data else None
        }

        if 'body_parts_mask' in data:
            resultado['body_parts_mask'] = data['body_parts_mask']

            grupos = {}
            for nombre_grupo in self.GRUPOS_PARTES_CUERPO.keys():
                key = f'grupo_{nombre_grupo}'
                if key in data:
                    grupos[nombre_grupo] = data[key]

            if grupos:
                resultado['grupos_partes'] = grupos

        return resultado

    def obtener_informacion_archivo(self, ruta_archivo: Path) -> Dict:
        """
        Obtiene informacion basica de un archivo NPZ.

        Args:
            ruta_archivo: Path al archivo NPZ

        Returns:
            Diccionario con metadatos del archivo

        Raises:
            FileNotFoundError: Si el archivo no existe
        """
        if not Path(ruta_archivo).exists():
            raise FileNotFoundError(f"Archivo no encontrado: {ruta_archivo}")

        data = np.load(ruta_archivo)

        info = {
            'nombre_archivo': Path(ruta_archivo).name,
            'dimensiones': data['person_mask'].shape if 'person_mask' in data else None,
            'threshold_value': float(data['threshold_value'][0]) if 'threshold_value' in data else None,
            'tiene_partes': 'body_parts_mask' in data,
            'arrays_disponibles': sorted(data.files),
            'tamaño_bytes': Path(ruta_archivo).stat().st_size
        }

        return info

    def listar_archivos(self) -> List[Path]:
        """
        Lista todos los archivos NPZ en el directorio.

        Returns:
            Lista de paths a archivos NPZ ordenados
        """
        return sorted(self.directorio_mascaras.glob("*.npz"))

    def obtener_estadisticas_directorio(self) -> Dict:
        """
        Obtiene estadisticas del directorio de mascaras.

        Returns:
            Diccionario con estadisticas del directorio
        """
        archivos = self.listar_archivos()

        if not archivos:
            return {
                'total_archivos': 0,
                'tamaño_total_mb': 0.0,
                'directorio': str(self.directorio_mascaras)
            }

        tamaño_total = sum(f.stat().st_size for f in archivos)

        return {
            'total_archivos': len(archivos),
            'tamaño_total_mb': round(tamaño_total / (1024 * 1024), 2),
            'directorio': str(self.directorio_mascaras)
        }

In [8]:
# =============================================================================
# GENERADOR DE VISUALIZACIONES
# =============================================================================

class GeneradorVisualizaciones:
    """Genera visualizaciones estandarizadas de resultados de segmentacion"""

    @staticmethod
    def generar_visualizacion(
        imagen: np.ndarray,
        mascara_probabilidad: np.ndarray,
        umbral_vis: float,
        ruta_salida: Path,
        titulo: str
    ):
        """
        Genera visualizacion de 2 paneles: imagen original y segmentacion.

        Args:
            imagen: Array RGB de la imagen original
            mascara_probabilidad: Mascara de probabilidad (0.0-1.0)
            umbral_vis: Umbral usado para la visualizacion
            ruta_salida: Path donde guardar la visualizacion
            titulo: Titulo para la figura
        """
        ruta_salida.parent.mkdir(parents=True, exist_ok=True)

        fig, axes = plt.subplots(1, 2, figsize=(16, 8))

        # Panel 1: Imagen original
        axes[0].imshow(imagen)
        axes[0].set_title('Imagen Original', fontsize=12, fontweight='bold')
        axes[0].axis('off')

        # Panel 2: Segmentacion con overlay
        axes[1].imshow(imagen)

        # Aplicar umbral para visualizacion
        mascara_binaria = mascara_probabilidad >= umbral_vis

        # Crear overlay verde semi-transparente para persona
        overlay = np.zeros_like(imagen)
        overlay[mascara_binaria] = [0, 255, 0]

        axes[1].imshow(overlay, alpha=0.4)
        axes[1].set_title(
            f'Segmentacion (umbral={umbral_vis})',
            fontsize=12,
            fontweight='bold'
        )
        axes[1].axis('off')

        # Calcular y mostrar estadisticas
        area_persona = mascara_binaria.sum()
        area_total = mascara_binaria.size
        porcentaje = (area_persona / area_total) * 100

        info_text = f"Area persona: {porcentaje:.1f}%\n"
        info_text += f"Pixels: {area_persona:,} / {area_total:,}"

        axes[1].text(
            0.02, 0.98, info_text,
            transform=axes[1].transAxes,
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
            fontsize=10,
            family='monospace'
        )

        plt.suptitle(titulo, fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(ruta_salida, dpi=100, bbox_inches='tight')
        plt.close()


In [9]:
class ProcesadorBodyPix:
    """
    Procesador para BodyPix con estructura de directorios estandarizada.

    MODIFICADO para consistencia con otros modelos del TFM:
    - Procesa UN umbral a la vez
    - Genera estructura bodypix/modelo/config_T{umbral}/
    - Guarda un archivo NPZ por imagen sin sufijo de umbral

    Procesa imagenes SIN redimensionar (BodyPix funciona mejor con imagenes grandes).
    Los valores de get_mask() son 0-1 (int32), NO 0-255.
    Incluye debug detallado de partes del cuerpo detectadas.
    """

    BODYPIX_PART_IDS = {
        -1: 'fondo',
        0: 'left_face',
        1: 'right_face',
        2: 'left_upper_arm_front',
        3: 'left_upper_arm_back',
        4: 'right_upper_arm_front',
        5: 'right_upper_arm_back',
        6: 'left_lower_arm_front',
        7: 'left_lower_arm_back',
        8: 'right_lower_arm_front',
        9: 'right_lower_arm_back',
        10: 'left_hand',
        11: 'right_hand',
        12: 'torso_front',
        13: 'torso_back',
        14: 'left_upper_leg_front',
        15: 'left_upper_leg_back',
        16: 'right_upper_leg_front',
        17: 'right_upper_leg_back',
        18: 'left_lower_leg_front',
        19: 'left_lower_leg_back',
        20: 'right_lower_leg_front',
        21: 'right_lower_leg_back',
        22: 'left_foot',
        23: 'right_foot'
    }

    def __init__(self, modelo_info: ModeloBodyPix, config: ConfiguracionEvaluacion):
        """
        Inicializa el procesador.

        Args:
            modelo_info: Informacion del modelo BodyPix
            config: Configuracion de evaluacion
        """
        self.modelo_info = modelo_info
        self.config = config
        self.model = None

    def cargar_modelo(self):
        """
        Descarga y carga el modelo BodyPix.
        """
        print(f"  Cargando modelo {self.modelo_info.nombre}...")
        model_path = download_model(self.modelo_info.path)
        self.model = load_model(model_path)
        print(f"  Modelo cargado correctamente")

    def procesar_imagen(
        self,
        img_path: Path,
        umbrales: List[float],
        ruta_salida: Path
    ) -> Tuple[Dict, np.ndarray, np.ndarray]:
        """
        Procesa una imagen con BodyPix.

        MODIFICADO: Ahora recibe UN solo umbral (lista con un elemento).
        Simplifica la logica de procesamiento para consistencia con otros modelos.

        Args:
            img_path: Path a la imagen a procesar
            umbrales: Lista con UN solo umbral
            ruta_salida: Directorio base para resultados

        Returns:
            Tupla (resultado_dict, imagen_array, mascara_probabilidad)

        Raises:
            ValueError: Si se reciben multiples umbrales

        CRITICO: NO redimensionar agresivamente. BodyPix funciona mejor con imagenes grandes.
        Los valores de get_mask() son 0-1 (int32), NO 0-255.
        """
        if len(umbrales) != 1:
            raise ValueError(
                f"ProcesadorBodyPix.procesar_imagen espera exactamente 1 umbral, "
                f"se recibieron {len(umbrales)}: {umbrales}"
            )

        umbral = umbrales[0]

        # Cargar imagen SIN redimensionar
        img_array, info_img = Utilidades.cargar_imagen(img_path, max_dimension=4096)

        print(f"    Input - Shape: {img_array.shape}, "
              f"Dtype: {img_array.dtype}, "
              f"Range: [{img_array.min()}, {img_array.max()}]")

        # Inferencia
        tiempo_inicio = time.time()

        result = self.model.predict_single(img_array)
        mask_raw = result.get_mask(threshold=umbral)

        # Convertir a numpy
        if hasattr(mask_raw, 'numpy'):
            mask_np = mask_raw.numpy()
        else:
            mask_np = np.array(mask_raw)

        # Shape (H, W, 1) -> (H, W)
        if mask_np.ndim == 3:
            mask_np = mask_np[:, :, 0]

        # Los valores YA son 0-1 (int32), NO dividir por 255
        mascara_probabilidad = mask_np.astype(np.float32)

        tiempo_inferencia_ms = (time.time() - tiempo_inicio) * 1000

        del result

        # Estadisticas
        area_persona = (mascara_probabilidad > 0.5).sum()
        area_total = mascara_probabilidad.size
        porcentaje = (area_persona / area_total) * 100

        print(f"    Mascara (umbral={umbral}) - "
              f"Area: {porcentaje:.1f}% "
              f"({area_persona:,} / {area_total:,} pixels)")

        # Partes del cuerpo
        mascara_partes = None
        try:
            result_parts = self.model.predict_single(img_array)
            mask_for_parts = result_parts.get_mask(threshold=umbral)

            if hasattr(mask_for_parts, 'numpy'):
                mask_for_parts = mask_for_parts.numpy()
            else:
                mask_for_parts = np.array(mask_for_parts)

            if mask_for_parts.ndim == 3:
                mask_for_parts = mask_for_parts[:, :, 0]

            # Valores son 0-1, convertir a 0-255 para get_part_mask
            mask_for_parts_255 = (mask_for_parts * 255).astype(np.uint8)

            parts_raw = result_parts.get_part_mask(mask_for_parts_255)

            if hasattr(parts_raw, 'numpy'):
                parts_raw = parts_raw.numpy()
            else:
                parts_raw = np.array(parts_raw)

            if parts_raw.ndim == 3:
                parts_raw = parts_raw[:, :, 0]

            mascara_partes = parts_raw.astype(np.int8)

            # Debug de partes detectadas
            self._debug_partes_detectadas(mascara_partes)

            del result_parts

        except Exception as e:
            print(f"    Sin partes del cuerpo: {str(e)}")

        # Guardar mascara
        gestor_mascaras = GestorMascaras(ruta_salida / "mascaras")
        archivo_npz = gestor_mascaras.guardar_mascaras(
            mascara_probabilidad,
            [umbral],
            img_path.name,
            {'nombre_corto': self.modelo_info.nombre_corto},
            mascara_partes=mascara_partes,
            masks_por_umbral={umbral: mascara_probabilidad}
        )

        # Calcular deteccion
        mascara_binaria = (mascara_probabilidad > 0.5).astype(np.uint8)
        area_persona_bin = int(mascara_binaria.sum())
        porcentaje_imagen = (area_persona_bin / area_total) * 100

        # Estadisticas de partes
        info_partes_cuerpo = None
        if mascara_partes is not None:
            info_partes_cuerpo = self._calcular_estadisticas_partes(
                mascara_partes,
                mascara_probabilidad
            )

        # Resultado
        resultado = {
            'metadata': {
                'imagen': {
                    'nombre': img_path.name,
                    'ruta': str(img_path),
                    **info_img
                },
                'modelo': {
                    'nombre': self.modelo_info.nombre,
                    'nombre_corto': self.modelo_info.nombre_corto,
                    'arquitectura': self.modelo_info.arquitectura,
                    'multiplicador': self.modelo_info.multiplicador
                },
                'configuracion': {
                    'umbral': umbral
                },
                'timestamp': datetime.now().isoformat(),
                'tiempo_inferencia_ms': round(tiempo_inferencia_ms, 2),
                'nota': 'BodyPix procesado con imagenes a resolucion completa'
            },
            'archivos_generados': {
                'mascara_npz': archivo_npz
            },
            'deteccion': {
                'persona_detectada': porcentaje_imagen >= 5.0,
                'area_pixels': area_persona_bin,
                'area_total_pixels': area_total,
                'porcentaje_imagen': round(porcentaje_imagen, 2),
                'probabilidad_media_region': float(
                    mascara_probabilidad[mascara_binaria > 0].mean()
                    if area_persona_bin > 0 else 0.0
                )
            },
            'partes_cuerpo': info_partes_cuerpo
        }

        return resultado, img_array, mascara_probabilidad

    def _debug_partes_detectadas(self, mascara_partes: np.ndarray):
        """
        Imprime debug detallado de las partes del cuerpo detectadas.

        Args:
            mascara_partes: Array (H, W) con IDs de partes del cuerpo
        """
        partes_unicas = np.unique(mascara_partes)
        area_total = mascara_partes.size

        print(f"    Partes detectadas ({len(partes_unicas)} tipos):")

        for part_id in partes_unicas:
            nombre = self.BODYPIX_PART_IDS.get(int(part_id), f'desconocida_{part_id}')
            area = (mascara_partes == part_id).sum()
            pct = (area / area_total) * 100

            # Solo mostrar partes significativas
            if pct > 0.1:
                print(f"      ID {part_id:2d}: {nombre:25s} - {pct:5.2f}% ({area:,} px)")

    def _calcular_estadisticas_partes(
        self,
        mascara_partes: np.ndarray,
        mascara_probabilidad: np.ndarray
    ) -> Dict:
        """
        Calcula estadisticas de partes del cuerpo detectadas.

        Retorna tanto estadisticas individuales como por grupos semanticos.

        Args:
            mascara_partes: Array (H, W) con IDs de partes
            mascara_probabilidad: Array (H, W) con probabilidades

        Returns:
            Diccionario con estadisticas detalladas de partes
        """
        # Normalizar dimensiones
        if mascara_probabilidad.ndim == 3:
            mascara_probabilidad = mascara_probabilidad[:, :, 0]

        if mascara_partes.ndim == 3:
            mascara_partes = mascara_partes[:, :, 0]

        area_total = mascara_partes.size

        # Estadisticas por parte individual
        partes_individuales = {}
        for part_id in np.unique(mascara_partes):
            if part_id == -1:
                continue

            mascara_parte = (mascara_partes == part_id)
            area_parte = int(mascara_parte.sum())

            if area_parte > 0:
                nombre = self.BODYPIX_PART_IDS.get(int(part_id), f'parte_{part_id}')
                prob_media = float(mascara_probabilidad[mascara_parte].mean())

                partes_individuales[nombre] = {
                    'id': int(part_id),
                    'area_pixels': area_parte,
                    'porcentaje_imagen': round((area_parte / area_total) * 100, 2),
                    'probabilidad_media': round(prob_media, 3)
                }

        # Estadisticas por grupos semanticos
        grupos_detectados = {}
        for nombre_grupo, ids_partes in GestorMascaras.GRUPOS_PARTES_CUERPO.items():
            mascara_grupo = np.isin(mascara_partes, ids_partes)
            area_grupo = int(mascara_grupo.sum())

            if area_grupo > 0:
                prob_media = float(mascara_probabilidad[mascara_grupo].mean())

                grupos_detectados[nombre_grupo] = {
                    'area_pixels': area_grupo,
                    'porcentaje_imagen': round((area_grupo / area_total) * 100, 2),
                    'probabilidad_media': round(prob_media, 3)
                }

        return {
            'partes_individuales': partes_individuales,
            'grupos_semanticos': grupos_detectados,
            'num_partes_detectadas': len(partes_individuales),
            'num_grupos_detectados': len(grupos_detectados),
            'grupos_disponibles': list(GestorMascaras.GRUPOS_PARTES_CUERPO.keys())
        }

    def limpiar_memoria(self):
        """
        Libera recursos del modelo y memoria.
        """
        if self.model is not None:
            del self.model
            self.model = None
        Utilidades.liberar_memoria()

In [10]:
class EvaluadorBodyPix:
    """
    Evaluador principal que coordina el procesamiento de modelos BodyPix.

    MODIFICADO para consistencia con otros modelos del TFM:
    - Crea directorios separados por umbral: config_T{umbral}/
    - Procesa cada umbral como configuracion independiente
    - Genera estructura compatible con indice maestro de Fase 1

    Estructura de salida:
    bodypix/
    └── mobilenet_v1_050/
        ├── ultra_sensible_T0_1/
        │   ├── mascaras/
        │   │   └── _DSC0036.npz
        │   ├── json/
        │   │   └── _DSC0036_timestamp.json
        │   └── visualizaciones/
        │       └── _DSC0036_timestamp.png
        ├── ultra_sensible_T0_15/
        │   └── ...
        └── ultra_sensible_T0_2/
            └── ...
    """

    def __init__(self, config: ConfiguracionEvaluacion):
        """
        Inicializa el evaluador.

        Args:
            config: Configuracion de evaluacion
        """
        self.config = config

    def evaluar_modelo_con_config_umbrales(
        self,
        modelo_key: str,
        config_umbral_nombre: str,
        imagenes: List[Path]
    ) -> Dict:
        """
        Evalua un modelo con una configuracion de umbrales especifica.

        MODIFICADO: Procesa cada umbral como configuracion separada.
        Crea un directorio independiente por cada umbral.

        Args:
            modelo_key: Identificador del modelo a evaluar
            config_umbral_nombre: Nombre de la configuracion de umbrales
            imagenes: Lista de paths a imagenes

        Returns:
            Diccionario con estadisticas del procesamiento
        """
        modelo_info = MODELOS_DISPONIBLES[modelo_key]
        umbrales_config = CONFIGURACIONES_UMBRALES[config_umbral_nombre]
        umbrales = umbrales_config.valores

        print(f"\n{'='*80}")
        print(f"MODELO: {modelo_info.nombre}")
        print(f"UMBRALES: {umbrales_config.nombre} - {umbrales}")
        print(f"{'='*80}")

        # Cargar modelo una vez para todos los umbrales
        procesador = ProcesadorBodyPix(modelo_info, self.config)
        procesador.cargar_modelo()

        total_procesadas = 0
        tiempo_total_inicio = time.time()

        # Procesar cada umbral como configuracion separada
        for idx_umbral, umbral in enumerate(umbrales, 1):
            # Crear nombre de configuracion unico por umbral
            umbral_str = str(umbral).replace('.', '_')
            config_nombre_con_umbral = f"{config_umbral_nombre}_T{umbral_str}"

            # Crear estructura de directorios por umbral
            ruta_salida_umbral = (
                self.config.ruta_salida /
                modelo_info.nombre_corto /
                config_nombre_con_umbral
            )

            print(f"\n{'#'*80}")
            print(f"UMBRAL {idx_umbral}/{len(umbrales)}: {umbral}")
            print(f"Configuracion: {config_nombre_con_umbral}")
            print(f"Directorio: {ruta_salida_umbral}")
            print(f"{'#'*80}")

            # Sistema de checkpoint por umbral
            checkpoint = GestorCheckpoint(ruta_salida_umbral / "checkpoint.json")
            nombres_imgs = [img.name for img in imagenes]
            pendientes = checkpoint.obtener_pendientes(nombres_imgs)
            imgs_procesar = [img for img in imagenes if img.name in pendientes]

            print(f"Progreso: {len(imagenes) - len(imgs_procesar)}/{len(imagenes)} completadas")
            print(f"Pendientes: {len(imgs_procesar)}")

            if len(imgs_procesar) == 0:
                print("Todas las imagenes ya procesadas para este umbral")
                continue

            # Procesar imagenes pendientes con este umbral
            for i, img_path in enumerate(imgs_procesar, 1):
                print(f"\n[{i}/{len(imgs_procesar)}] {img_path.name}")

                try:
                    # Procesar imagen con UN SOLO umbral
                    resultado, img_array, mascara_prob = procesador.procesar_imagen(
                        img_path,
                        [umbral],  # Lista con un solo umbral
                        ruta_salida_umbral
                    )

                    # Guardar JSON
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    nombre_json = f"{img_path.stem}_{timestamp}.json"
                    ruta_json = ruta_salida_umbral / "json" / nombre_json
                    Utilidades.guardar_json(resultado, ruta_json)

                    # Generar visualizacion
                    nombre_vis = f"{img_path.stem}_{timestamp}.png"
                    ruta_vis = ruta_salida_umbral / "visualizaciones" / nombre_vis

                    GeneradorVisualizaciones.generar_visualizacion(
                        img_array,
                        mascara_prob,
                        umbral,
                        ruta_vis,
                        f"{modelo_info.nombre_corto} - {config_nombre_con_umbral}"
                    )

                    # Marcar como completada
                    checkpoint.marcar_completada(img_path.name)

                    # Resumen
                    persona_det = resultado['deteccion']['persona_detectada']
                    print(f"  Completada - Persona detectada: {'Si' if persona_det else 'No'}")

                    total_procesadas += 1

                except Exception as e:
                    print(f"  ERROR: {str(e)}")
                    import traceback
                    traceback.print_exc()

                finally:
                    Utilidades.liberar_memoria()
                    if i < len(imgs_procesar):
                        time.sleep(self.config.pausa_entre_imagenes)

        tiempo_total = time.time() - tiempo_total_inicio

        # Limpiar modelo
        procesador.limpiar_memoria()

        print(f"\nConfiguracion completada en {tiempo_total:.1f}s")
        print(f"Total imagenes procesadas: {total_procesadas}")

        return {
            'procesadas': total_procesadas,
            'umbrales_procesados': len(umbrales),
            'tiempo_segundos': round(tiempo_total, 2)
        }

    def ejecutar(self):
        """
        Ejecuta la evaluacion completa de todos los modelos y configuraciones.

        Coordina el procesamiento de todas las combinaciones de:
        - Modelos (ej: mobilenet_v1_050, mobilenet_v1_075, mobilenet_v1_100)
        - Configuraciones de umbrales (ej: ultra_sensible, sensibilidad_alta, etc.)
        - Umbrales dentro de cada configuracion
        """
        print(f"\n{'='*80}")
        print("EVALUADOR BODYPIX")
        print(f"{'='*80}")
        print(f"Modelos: {len(self.config.modelos_evaluar)}")
        for modelo_key in self.config.modelos_evaluar:
            modelo = MODELOS_DISPONIBLES[modelo_key]
            print(f"  - {modelo.nombre} (multiplicador {modelo.multiplicador})")

        print(f"\nConfiguraciones de umbrales: {len(self.config.configs_umbrales)}")
        for config_name in self.config.configs_umbrales:
            umbrales = CONFIGURACIONES_UMBRALES[config_name]
            print(f"  - {config_name}: {umbrales.valores}")

        print(f"\nDirectorio de imagenes: {self.config.ruta_imagenes}")
        print(f"Directorio de resultados: {self.config.ruta_salida}")

        # Cargar imagenes
        imagenes = []
        for extension in ['*.jpg', '*.JPG', '*.jpeg', '*.JPEG', '*.png', '*.PNG']:
            imagenes.extend(sorted(self.config.ruta_imagenes.glob(extension)))
        imagenes = sorted(set(imagenes))

        print(f"\nTotal imagenes encontradas: {len(imagenes)}")

        if not imagenes:
            print("ERROR: No se encontraron imagenes en el directorio")
            return

        # Evaluar cada combinacion de modelo + configuracion
        total_combinaciones = (
            len(self.config.modelos_evaluar) *
            len(self.config.configs_umbrales)
        )
        combinacion_actual = 0

        for modelo_key in self.config.modelos_evaluar:
            for config_umbral in self.config.configs_umbrales:
                combinacion_actual += 1

                print(f"\n{'#'*80}")
                print(f"COMBINACION {combinacion_actual}/{total_combinaciones}")
                print(f"{'#'*80}")

                self.evaluar_modelo_con_config_umbrales(
                    modelo_key,
                    config_umbral,
                    imagenes
                )

                if combinacion_actual < total_combinaciones:
                    print(f"\nPausa entre configuraciones: {self.config.pausa_entre_modelos}s")
                    time.sleep(self.config.pausa_entre_modelos)

        print(f"\n{'='*80}")
        print("EVALUACION COMPLETADA")
        print(f"{'='*80}")
        print(f"Resultados guardados en: {self.config.ruta_salida}")
        print(f"\nEstructura de salida:")
        print(f"  bodypix/")
        print(f"  └── modelo/")
        print(f"      └── config_T{{umbral}}/")
        print(f"          ├── mascaras/{{foto}}.npz")
        print(f"          ├── json/{{foto}}_{{timestamp}}.json")
        print(f"          └── visualizaciones/{{foto}}_{{timestamp}}.png")

In [11]:
# =============================================================================
# EJECUCION PRINCIPAL
# =============================================================================

def main():
    # Configurar evaluacion
    config = ConfiguracionEvaluacion(
        # Seleccionar [modelos] (None = todos los 3)
        modelos_evaluar= None,

        # Seleccionar configuraciones de umbrales
        configs_umbrales=[
            'ultra_sensible',
            'sensibilidad_alta',
            'sensibilidad_media',
            'baja_sensibilidad'
        ],

        # Parametros de procesamiento
        max_dimension=1024,
        pausa_entre_imagenes=2.0,
        pausa_entre_modelos=5.0
    )

    # Ejecutar evaluacion
    evaluador = EvaluadorBodyPix(config)
    evaluador.ejecutar()

In [12]:
if __name__ == "__main__":
    main()


EVALUADOR BODYPIX
Modelos: 2
  - MobileNetV1 0.50 (multiplicador 0.5)
  - MobileNetV1 0.75 (multiplicador 0.75)

Configuraciones de umbrales: 4
  - ultra_sensible: [0.1, 0.15, 0.2]
  - sensibilidad_alta: [0.15, 0.2, 0.25]
  - sensibilidad_media: [0.2, 0.3, 0.4]
  - baja_sensibilidad: [0.3, 0.4, 0.5]

Directorio de imagenes: /content/drive/MyDrive/TFM/0_Imagenes
Directorio de resultados: /content/drive/MyDrive/TFM/2_Modelos/bodypix

Total imagenes encontradas: 20

################################################################################
COMBINACION 1/8
################################################################################

MODELO: MobileNetV1 0.50
UMBRALES: ultra_sensible - [0.1, 0.15, 0.2]
  Cargando modelo MobileNetV1 0.50...
  Modelo cargado correctamente

################################################################################
UMBRAL 1/3: 0.1
Configuracion: ultra_sensible_T0_1
Directorio: /content/drive/MyDrive/TFM/2_Modelos/bodypix/mobilenet_v1_050/ultra_s

In [13]:
# ==============================================================================
# TEST DIAGNÓSTICO AISLADO - EJECUTA ESTO EN UNA CELDA SEPARADA
# ==============================================================================

# from tf_bodypix.api import download_model, load_model, BodyPixModelPaths
# import cv2
# import numpy as np

# # Cargar modelo
# print("Cargando modelo...")
# model_path = download_model(BodyPixModelPaths.MOBILENET_FLOAT_50_STRIDE_16)
# model = load_model(model_path)
# print("Modelo cargado\n")

# img_path = "/content/drive/MyDrive/TFM/0_Imagenes/_DSC0023.jpg"
# print(f"Cargando imagen: {img_path}")

# img_bgr = cv2.imread(img_path)
# img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

# print(f"Imagen: {img_rgb.shape}, {img_rgb.dtype}, [{img_rgb.min()}, {img_rgb.max()}]\n")

# # Inferencia
# print("Haciendo inferencia...")
# result = model.predict_single(img_rgb)
# print("Inferencia completada\n")

# print("Resultados por threshold:")
# print("-" * 60)

# for t in [0.05, 0.1, 0.15, 0.2, 0.3, 0.5]:
#     mask = result.get_mask(threshold=t)

#     # Verificar tipo
#     print(f"\nThreshold {t}:")
#     print(f"  Tipo de mask: {type(mask)}")

#     # Convertir a numpy
#     if hasattr(mask, 'numpy'):
#         mask_np = mask.numpy()
#     else:
#         mask_np = np.array(mask)

#     print(f"  Shape: {mask_np.shape}")
#     print(f"  Dtype: {mask_np.dtype}")
#     print(f"  Range: [{mask_np.min()}, {mask_np.max()}]")

#     # Asegurar 2D
#     if mask_np.ndim == 3:
#         mask_np = mask_np[:, :, 0]

#     # Contar pixels
#     area = (mask_np > 0).sum()
#     total = mask_np.size
#     pct = (area / total) * 100

#     print(f"  Pixels detectados: {area:,} ({pct:.1f}%)")
#     print(f"  Valores únicos: {np.unique(mask_np)}")

# print("\n" + "=" * 60)